In [ ]:
#Download
     #tqdm
     #torchvision
     #cv2   
#Pytorch dataset 
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from astropy.io import fits
import numpy as np
import random

#Deep Learning
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.utils import save_image
#images#
import cv2
import os
from matplotlib import pyplot as plt

#sesion save# 
import dill

#
from datasetImages import DatasetImages
from datasetDirty import DatasetDirty
from datasetPSF import DatasetPSF
import cupy as cp
from paramsEllipses import ParamsEllipses


In [ ]:
N = 28 # image size
N_PSF = 28 # psf size
TYPE_PSF = 'psf_gauss_'+str(N)+'x'+str(N)
# **Params Dataset Python**
PATH_IMAGES = '../datasets/images_'+str(N)+'x'+str(N)+'/images' #path dataset images
PATH_CONVOLUTION = '../datasets/images_'+str(N)+'x'+str(N)+'/convolutions/'+TYPE_PSF+'/conv'#path dataset convolution
PATH_PSF_SAVE =  '../datasets/images_'+str(N)+'x'+str(N)+'/convolutions/'+TYPE_PSF+'/psf'#path convolution
INITIAL_DATASET = 0  #initial index for the names of the saved images 
FINAL_DATASET =   10000#final index for the names of the saved images 
STEP_DATASET = 1000
PERC_TRAIN = 0.7 # (70%) #train percentage 
PERC_VAL = 0.3 #  (25#) #validation percentage
#PERC_TEST =  0.05 # (5%) #test percentag

BATCH_TRAIN = 10
BATCH_VALIDATION = 10 #batch sizeimages
BATCH_TEST = 1

NUM_EPOCHS = 10
LEARNING_RATE = 1e-3
size_data= FINAL_DATASET - INITIAL_DATASET
PATH_VALIDATION =  '../datasets/images_'+str(N)+'x'+str(N)+'/convolutions/'+TYPE_PSF+'/models/model_S'+str(size_data)+'_B'+str(BATCH_TRAIN)+'_N'+str(NUM_EPOCHS)+'/validation'
PATH_TEST =  '../datasets/images_'+str(N)+'x'+str(N)+'/convolutions/'+TYPE_PSF+'/models/model_S'+str(size_data)+'_B'+str(BATCH_TRAIN)+'_N'+str(NUM_EPOCHS)+'/test'
PATH_GRAPH ='../datasets/images_'+str(N)+'x'+str(N)+'/convolutions/'+TYPE_PSF+'/models/model_S'+str(size_data)+'_B'+str(BATCH_TRAIN)+'_N'+str(NUM_EPOCHS)+'/graph'
PATH_MODEL_SAVE ='../datasets/images_'+str(N)+'x'+str(N)+'/convolutions/'+TYPE_PSF+'/models/model_S'+str(size_data)+'_B'+str(BATCH_TRAIN)+'_N'+str(NUM_EPOCHS)+'/model' #path where the dataset convolution is saved

In [ ]:
def save_decoded_image(img, name):
    img = img.view(img.size(0), 1, N, N)
    save_image(img, name)
    
def make_dir(path):
        if not os.path.exists(path):
            os.makedirs(path)

def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device


In [ ]:
tsfms=transforms.Compose([
    transforms.ToTensor()
])


class interferometryDataset(Dataset):
  def __init__(self,datasetnoised,datasetclean,transform):
    self.noise=datasetnoised
    self.clean=datasetclean
    self.transform=transform
  
  def __len__(self):
    return len(self.noise)
  
  def __getitem__(self,idx):
    xNoise=self.noise[idx]
    xClean=self.clean[idx]
    if self.transform != None:
      xNoise=self.transform(cp.asnumpy(xNoise))
      xClean=self.transform(cp.asnumpy(xClean))

    return (xNoise,xClean)

In [ ]:
def train_data_arange(index):

    size = STEP_DATASET
    size_train = round(size*PERC_TRAIN)
    batch_train_size=  BATCH_TRAIN 

    params = ParamsEllipses(N)

    data_image = DatasetImages(N)
    data_image.create(size_image=N,params = params , start = index,stop = index +size_train)

    data_psf = DatasetPSF(N,TYPE_PSF)
    data_psf.create(N,TYPE_PSF)

    data_dirty = DatasetDirty(N,TYPE_PSF,data_psf.image)
    data_dirty.create(images = data_image.images,size_image = N, type_psf = TYPE_PSF, psf =data_psf.image)
    trainSet=interferometryDataset(data_dirty.dirtys,data_image.images,tsfms)
    trainLoader=DataLoader(trainSet,batch_train_size,shuffle=True)
    return trainLoader


def validation_data_arange(index):
    size = STEP_DATASET
    index = index + round(size*PERC_TRAIN)
    size_validation =  round(size*PERC_VAL)
    batch_validation_size=  BATCH_VALIDATION

    params = ParamsEllipses(N)

    data_image = DatasetImages(N)
    data_image.create(size_image=N,params = params , start = index,stop = index +size_validation)

    data_psf = DatasetPSF(N,TYPE_PSF)
    data_psf.create(N,TYPE_PSF)

    data_dirty = DatasetDirty(N,TYPE_PSF,data_psf.image)
    data_dirty.create(images = data_image.images,size_image = N, type_psf = TYPE_PSF, psf =data_psf.image)
    validationSet = interferometryDataset(data_dirty.dirtys,data_image.images,tsfms)
    validationLoader=DataLoader(validationSet,batch_validation_size,shuffle=True)
    return validationLoader

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # encoder layers
        self.enc1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.enc2 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.enc3 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.enc4 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        # decoder layers
        self.dec1 = nn.ConvTranspose2d(8, 8, kernel_size=3, stride=2)  
        self.dec2 = nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2)
        self.dec3 = nn.ConvTranspose2d(16, 32, kernel_size=2, stride=2)
        self.dec4 = nn.ConvTranspose2d(32, 64, kernel_size=2, stride=2)
        self.out = nn.Conv2d(64, 1, kernel_size=3, padding=1)
    def forward(self, x):
        # encode
        x = F.relu(self.enc1(x))
        x = self.pool(x)
        x = F.relu(self.enc2(x))
        x = self.pool(x)
        x = F.relu(self.enc3(x))
        x = self.pool(x)
        x = F.relu(self.enc4(x))
        x = self.pool(x) # the latent space representation
        
        # decode
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        x = F.sigmoid(self.out(x))
        return x
net = Autoencoder()

In [ ]:
criterion = nn.MSELoss()
# the optimizaater
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)



In [ ]:

def train(net, NUM_EPOCHS,path):
  train_loss = []
  valid_loss = []
  list_index = cp.arange(INITIAL_DATASET,FINAL_DATASET,STEP_DATASET)
  for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    size_train = 0
    for index in list_index:
        trainLoader = train_data_arange(index)
        size_train = size_train +len(trainLoader)
        print('train size:'+str(size_train))
        for dirty,clean in tqdm((trainLoader)):
          dirty,clean=dirty.to(device).float(),clean.to(device).float()

          optimizer.zero_grad()            
          outputs = net(dirty)
          loss = criterion(outputs, clean)
          # backpropagation
          loss.backward()
          # update the parameters
          optimizer.step()
          running_loss += loss.item()
    loss = running_loss / size_train
    train_loss.append(loss)
    
      ## validation ## '''
    validation_loss = 0.0
    net.eval()
    size_validate = 0  
    for index in list_index:
        validationloader = validation_data_arange(index)
        size_validate = size_validate + len(validationloader)
        print('validate size:'+str(size_validate))
        for dirty, clean in validationloader:
            dirty,clean=dirty.to(device).float(),clean.to(device).float()
            optimizer.zero_grad()            
            outputs = net(dirty)
            loss = criterion(outputs, clean)
            validation_loss += loss.item()
    loss = validation_loss / size_validate
    valid_loss.append(loss)
    print('Epoch {} of {}, Train Loss: {:.3f}'.format(
        epoch+1, NUM_EPOCHS, loss))
    '''save_decoded_image(dirty.cpu().data, name='./'+path+'/noisy{}.png'.format(epoch))
    save_decoded_image(outputs.cpu().data, name='./'+path+'/denoised{}.png'.format(epoch))
    save_decoded_image(clean.cpu().data, name='./'+path+'/clean{}.png'.format(epoch))'''
  return net,train_loss,valid_loss

In [ ]:
torch.cuda.empty_cache()
device = get_device()
print(device)
net.to(device)
make_dir(PATH_VALIDATION)
make_dir(PATH_GRAPH)

net,train_loss,validation_loss = train(net, NUM_EPOCHS,PATH_VALIDATION)

In [ ]:
plt.figure()
plt.plot(train_loss)
plt.plot(validation_loss)
plt.title('Train Loss & Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.savefig(PATH_GRAPH+'/graph_loss_train_validation.png')
